DMTOOLS External API Test and Demo - 29 July 2025

Notebook to test library that has been published tp pypi test

In [1]:
!pip install python-dotenv
!pip install matplotlib
!pip install plotly==5.23.0
!pip install -i https://test.pypi.org/simple/ dmtools-brown-edu --upgrade

In [3]:
## need to disconnect and delete the runtime and rerun the notebook to reinstall this properly
## check that it says it has sucessfully installed
!pip install -i https://test.pypi.org/simple/ dmtools-brown-edu --upgrade

Looking in indexes: https://test.pypi.org/simple/


In [4]:
import json
import os

In [5]:
import json
from dotenv import load_dotenv
import os

def load_secrets(env_file_path=".env"):
    load_dotenv(env_file_path)
    email = os.getenv('EMAIL')
    api_key = os.getenv('API_KEY')
    if not email or not api_key:
        missing = []
        if not email:
            missing.append('EMAIL')
        if not api_key:
            missing.append('API_KEY')
        raise KeyError(f"Missing required secrets: {', '.join(missing)}")
    return {"email": email, "api_key": api_key}
cwd = os.getcwd()
print("Current working directory:", cwd)
secrets = load_secrets("./secrets/.env")

In [6]:
cwd = os.getcwd()
print("Current working directory:", cwd)

Current working directory: C:\Users\devuser\Code\dmtools_notebooks\Demos


In [7]:
secrets = load_secrets("./secrets/.env")
#print(secrets["email"])
#print(secrets["api_key"])

In [8]:
from dmtools_brown_edu.dmtools_client_package.dmtools_quart import APIClient

In [9]:
## You need to enter your email address into the colab secrets
## see key in left hand bar
## The library cleans the email address provided
## First time you run this you need to give permission to the notebook to access each of the secrets
#from google.colab import userdata
# email = userdata.get('DMTOOLS_EMAIL')
# print(email)

In [10]:
#api_key = userdata.get('DMTOOLS_APIKEY')

In [11]:
email = secrets["email"]
api_key = secrets["api_key"]
print(email)
Client = APIClient(email, api_key, suppress_logs=False)

In [12]:
print(email)

andrewgaitskell@brown.edu


Create the API Client - this connects to the server and allows simpler interactions.

For less verbose calling, you can set suppress_logs=True

In [13]:
Client = APIClient(email, api_key, suppress_logs=False)

Query using key pair - full match on word
Requirement was to exact word match for the key value pair

Example match to CDMS but not to CDMSSuper

Below returns a list and the results need to be loaded as json

In [14]:
r = Client.read(query="experiment:CDMS")
record_count = len(r)
experiment_list = []

2025-07-30 06:43:08,616 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API


In [15]:
r

[[{'archived': 'Thu, 23 Jan 2025 21:35:03 GMT',
   'created': 'Tue, 21 Jan 2025 14:22:36 GMT',
   'created_by_user_id': 2,
   'experiment': 'CDMS II (Soudan)',
   'id': 15,
   'name': ' CDMS II (Soudan), projection 2004, SD-proton ',
   'resulttype': 'Proj',
   'type': 'Data',
   'updated': 'Thu, 23 Jan 2025 21:35:03 GMT',
   'year': '2004'},
  {'comment': 'Spin-dependent neutron cross section',
   'dateAnnouncement': '2004-01-01',
   'dateEnd': None,
   'dateOfficial': None,
   'dateStart': None,
   'defaultColor': 'blue',
   'defaultStyle': 'dot',
   'experiment': 'CDMS II (Soudan)',
   'greatestHit': 0,
   'label': 'CDMS II (Soudan), projection 2004, SD-proton',
   'label_short': 'CDMS II (Soudan), projection 2004, SD-proton',
   'measurementType': 'Dir',
   'official': 1,
   'open': 0,
   'public': 1,
   'rating': 0,
   'raw': '{[6.91830970918936 39044434194.6128;7.76247116628692 4606.46598010512;8.70963589956081 362.771262854754;9.77237220955811 69.0217904343617;10.9647819614319 1

In [16]:
for rc in range(0, record_count):
  r_p = r[rc][1]
  r_p_experiment = r_p['experiment']
  experiment_list.append(r_p_experiment)


In [17]:
unique_list = list(set(experiment_list))
print(unique_list)

['CDMS I (SUF)', 'CDMS II (Soudan)']


Read a Single record by the ID
Below returns a json variable in a list - the above result from a query needs to be updated to also return a list of json variables

In [18]:
r_id = Client.read(id=1)
r_id[0]
properties_json = r_id[0][1]
#properties_json

2025-07-30 06:43:10,072 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API


In [19]:
r_id[0][0]['archived']

In [20]:
archive_via_id = Client.archive(id=1)

2025-07-30 06:43:10,526 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling archive data API


In [21]:
archive_via_id

[{'id': 1, 'message': 'Node archived'}, 200]

In [22]:
r_id = Client.read(id=1)
r_id[0][0]

2025-07-30 06:43:13,901 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API


{'archived': 'Wed, 30 Jul 2025 05:43:11 GMT',
 'created': 'Tue, 21 Jan 2025 14:22:36 GMT',
 'id': 1,
 'type': 'Data',
 'updated': 'Wed, 30 Jul 2025 05:43:11 GMT'}

In [23]:
reverse_archive_via_id = Client.archive_reverse(id=1)

2025-07-30 06:43:14,459 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling remove archive data API


In [24]:
reverse_archive_via_id

[{'id': 1, 'message': 'Node archival reversed'}, 200]

In [25]:
r_id = Client.read(id=1)
r_id[0][0]

2025-07-30 06:43:14,906 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API


{'archived': None,
 'created': 'Tue, 21 Jan 2025 14:22:36 GMT',
 'id': 1,
 'type': 'Data',
 'updated': 'Wed, 30 Jul 2025 05:43:11 GMT'}

In [26]:
#new_properties_json = json.loads(r_id)
new_properties = r_id[0][1]
type(new_properties)

dict

In [27]:
new_properties_json = new_properties
#new_properties_json

In [28]:
new_data = Client.create(data=new_properties_json,node_type='Data')

2025-07-30 06:43:15,375 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling add data API
2025-07-30 06:43:15,714 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - API Response: 201 - [{"archived":null,"created":"Wed, 30 Jul 2025 05:43:16 GMT","id":16966,"type":"Data","updated":"Wed, 30 Jul 2025 05:43:16 GMT"},{"comment":" ","dateAnnouncement":"2003-01-01","dateEnd":null,"dateOfficial":null,"dateStart":null,"defaultColor":"red","defaultStyle":"dash","experiment":"Theory","greatestHit":1,"label":"Baer et. al., 2003, constraints on mSUGRA, SI","label_short":"Baer et. al., 2003, constraints on mSUGRA, SI","measurementType":"Dir","official":1,"open":0,"public":1,"rating":0,"raw":"{[60 4e-43;70 4e-43;160 5.5e-44;320 2.5e-44;630 3.2e-44;640 3e-44;640 1.2e-44;630 1e-44;225 9e-45;215 7e-45;315 7e-46;470 2.1e-46;650 7e-47;660 6.8e-47;660 2e-47;650 2e-47;470 6e-47;315 1e-46;215 3e-46;100 8e-46;60 1e-45;60 4e-43]}","reference":"JCAP 0309 (2003) 007 , hep-ph/03

create api data response >>>>>>> 201 [{"archived":null,"created":"Wed, 30 Jul 2025 05:43:16 GMT","id":16966,"type":"Data","updated":"Wed, 30 Jul 2025 05:43:16 GMT"},{"comment":" ","dateAnnouncement":"2003-01-01","dateEnd":null,"dateOfficial":null,"dateStart":null,"defaultColor":"red","defaultStyle":"dash","experiment":"Theory","greatestHit":1,"label":"Baer et. al., 2003, constraints on mSUGRA, SI","label_short":"Baer et. al., 2003, constraints on mSUGRA, SI","measurementType":"Dir","official":1,"open":0,"public":1,"rating":0,"raw":"{[60 4e-43;70 4e-43;160 5.5e-44;320 2.5e-44;630 3.2e-44;640 3e-44;640 1.2e-44;630 1e-44;225 9e-45;215 7e-45;315 7e-46;470 2.1e-46;650 7e-47;660 6.8e-47;660 2e-47;650 2e-47;470 6e-47;315 1e-46;215 3e-46;100 8e-46;60 1e-45;60 4e-43]}","reference":"JCAP 0309 (2003) 007 , hep-ph/0305191","resultType":"Th","spinDependency":"SI","values":[[["6.00000e+01","4.00000e-43"],["7.00000e+01","4.00000e-43"],["1.60000e+02","5.50000e-44"],["3.20000e+02","2.50000e-44"],["6.30

In [29]:
new_data

{'status_code': 201,
 'message': [{'archived': None,
   'created': 'Wed, 30 Jul 2025 05:43:16 GMT',
   'id': 16966,
   'type': 'Data',
   'updated': 'Wed, 30 Jul 2025 05:43:16 GMT'},
  {'comment': ' ',
   'dateAnnouncement': '2003-01-01',
   'dateEnd': None,
   'dateOfficial': None,
   'dateStart': None,
   'defaultColor': 'red',
   'defaultStyle': 'dash',
   'experiment': 'Theory',
   'greatestHit': 1,
   'label': 'Baer et. al., 2003, constraints on mSUGRA, SI',
   'label_short': 'Baer et. al., 2003, constraints on mSUGRA, SI',
   'measurementType': 'Dir',
   'official': 1,
   'open': 0,
   'public': 1,
   'rating': 0,
   'raw': '{[60 4e-43;70 4e-43;160 5.5e-44;320 2.5e-44;630 3.2e-44;640 3e-44;640 1.2e-44;630 1e-44;225 9e-45;215 7e-45;315 7e-46;470 2.1e-46;650 7e-47;660 6.8e-47;660 2e-47;650 2e-47;470 6e-47;315 1e-46;215 3e-46;100 8e-46;60 1e-45;60 4e-43]}',
   'reference': 'JCAP 0309 (2003) 007 , hep-ph/0305191',
   'resultType': 'Th',
   'spinDependency': 'SI',
   'values': [[['6.0

In [30]:
new_properties_json['rating']=1000

In [31]:
update_data = Client.update(id=16805,updated_data=new_properties_json,node_type='Data')

2025-07-30 06:43:15,742 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling update data API
2025-07-30 06:43:16,087 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - WARNING - HTTP error response: 500 - {'error': 'cannot unpack non-iterable NoneType object'}


In [32]:
update_data

{'success': False,
 'status_code': 500,
 'error': 'cannot unpack non-iterable NoneType object'}

In [37]:
r_id = Client.read(id=16805)
r_id

2025-07-30 06:46:27,114 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API


[[{'created': '2025-07-30T05:46:30.846Z',
   'id': -1,
   'type': 'Data',
   'updated': '2025-07-30T05:46:30.846Z'},
  {'comment': 'comment',
   'dateAnnouncement': None,
   'dateEnd': None,
   'dateOfficial': None,
   'dateStart': None,
   'defaultColor': 'black',
   'defaultStyle': 'line',
   'experiment': 'Experiment',
   'greatestHit': 0,
   'hepUrl': 'https://www.hepdata.net/record/show/hepdata.123456',
   'label': 'label',
   'label_short': 'label',
   'measurementType': 'Unknown',
   'official': 0,
   'open': 0,
   'public': 0,
   'rating': 0,
   'raw': "[[['0','0'],['0','1'],['1','1'],['1','0'],['0','0']]]",
   'reference': 'reference',
   'resultType': 'Th',
   'spinDependency': 'SD',
   'values': [[['0', '0'], ['0', '1'], ['1', '1'], ['1', '0'], ['0', '0']]],
   'xRescale': '1',
   'xUnits': 'GeV/c^2',
   'yRescale': '1',
   'yUnits': 'cm2',
   'year': 2000}]]

In [34]:
r_id = Client.delete(id=16805)

2025-07-30 06:43:16,450 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling delete data API
2025-07-30 06:43:16,794 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - WARNING - HTTP error response: 500 - {'error': "'id'"}


In [35]:
r_id = Client.read(id=16805)

2025-07-30 06:43:16,804 - dmtools_brown_edu.dmtools_client_package.dmtools_quart - INFO - Calling read data API


In [36]:
r_id

[[{'created': '2025-07-30T05:43:17.591Z',
   'id': -1,
   'type': 'Data',
   'updated': '2025-07-30T05:43:17.591Z'},
  {'comment': 'comment',
   'dateAnnouncement': None,
   'dateEnd': None,
   'dateOfficial': None,
   'dateStart': None,
   'defaultColor': 'black',
   'defaultStyle': 'line',
   'experiment': 'Experiment',
   'greatestHit': 0,
   'hepUrl': 'https://www.hepdata.net/record/show/hepdata.123456',
   'label': 'label',
   'label_short': 'label',
   'measurementType': 'Unknown',
   'official': 0,
   'open': 0,
   'public': 0,
   'rating': 0,
   'raw': "[[['0','0'],['0','1'],['1','1'],['1','0'],['0','0']]]",
   'reference': 'reference',
   'resultType': 'Th',
   'spinDependency': 'SD',
   'values': [[['0', '0'], ['0', '1'], ['1', '1'], ['1', '0'], ['0', '0']]],
   'xRescale': '1',
   'xUnits': 'GeV/c^2',
   'yRescale': '1',
   'yUnits': 'cm2',
   'year': 2000}]]